In [1]:
# ! pip install transformers datasets bitsandbytes 

### Phi-3 family of models 
 - Come in `phi-3-mini`(3B) & `phi-3-small`(7B) variants
 - The 'mini' version can be run on a Mobile phone.
 - The `phi-3-mini` version uses the transformer decoder architecture, using same layers like Llama model.
 - The tokenizer used is tiktokenizer with similar vocab like llama.

In [2]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig, AutoPeftModelForCausalLM

_ = torch.manual_seed(0)


#### Load the model

In [3]:
with open('secrets/secrets.json', 'r') as f:
    secrets = json.load(f)

HF_TOKEN = secrets['HF_TOKEN']
WANDB_TOKEN = secrets['WANDB_TOKEN']

In [4]:
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-3-mini-128k-Instruct",
    quantization_config=config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    attn_implementation="flash_attention_2",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# phi tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-3-mini-128k-Instruct')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
message = [
    {"role": "system", "content": "You are a helpful chat assistant."},
    {"role": "user", "content": "What are Large Language Models ?"},
]
tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True,
)

'<s><|system|>\nYou are a helpful chat assistant.<|end|>\n<|user|>\nWhat are Large Language Models ?<|end|>\n<|assistant|>\n'

In [7]:
tokenizer.save_pretrained('tokenizers/microsoft/phi-3-mini-128k-Instruct')

('tokenizers/microsoft/phi-3-mini-128k-Instruct/tokenizer_config.json',
 'tokenizers/microsoft/phi-3-mini-128k-Instruct/special_tokens_map.json',
 'tokenizers/microsoft/phi-3-mini-128k-Instruct/tokenizer.model',
 'tokenizers/microsoft/phi-3-mini-128k-Instruct/added_tokens.json',
 'tokenizers/microsoft/phi-3-mini-128k-Instruct/tokenizer.json')

### Training the model
 - SFT Tuning the phi-3-mini-Instruct-128k model to follow finance instructions.
 

In [ ]:
# NOTE: Decide on the dataset used, reference papers "FinGPT & BloombergGPT"

### Inference on the model

In [10]:
from transformers import pipeline, GenerationConfig

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)

prompts = [
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "It's Bengay for muscle relief, a combination of methyl salicylate, menthol, and what other active ingredient commonly found in aspirin?",
    "How can i get rid of llamas in my backyard?",
]

messages = []
for prompt in prompts:
    messages.append(
        [
        {'role': 'system', 'content': 'You are a helpul chat assistant.'},
        {'role': 'user', 'content': prompt}
        ]
    )

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

generations = pipe(messages, **generation_args)


/home/ostrich/anaconda3/envs/my_master/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [18]:
# display the results
for i, generation in enumerate(generations):
    print("=" * 25)
    print('PROMPT:')
    print(f'{prompts[i]}')
    print("=" * 25)

    print('GENERATION:')
    print(f"{generation[0]['generated_text']}")
    print("=" * 25)

PROMPT:
A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?
GENERATION:
To find out how much fencing is needed, you need to calculate the perimeter of the rectangular garden. The formula for the perimeter of a rectangle is 2*(length + width). 

So, in this case, the calculation would be: 2 * (25 feet + 15 feet) = 2 * 40 feet = 80 feet.

Therefore, you would need 80 feet of fencing to surround the entire garden.
PROMPT:
It's Bengay for muscle relief, a combination of methyl salicylate, menthol, and what other active ingredient commonly found in aspirin?
GENERATION:
The active ingredient commonly found in aspirin that is also a component of Bengay for muscle relief is acetaminophen.
PROMPT:
How can i get rid of llamas in my backyard?
GENERATION:
If you're finding llamas in your backyard, it's likely a misunderstanding as llamas are not common household pets and are native to So